# CHAPTER 1 Supervised Learning

Include：K-Fold Cross Validation, KNN, Naive Bayes, Linear Regression, Polynomial Regression, Perceptron, Decision Tree, Random Forest

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. K-Fold Cross Validation
K折交叉验证（K-Fold Cross Validation）是一种常用的模型评估方法。它的基本思想是将训练集分成K个子集，然后将每个子集依次作为验证集，其余的K-1个子集作为训练集，这样会得到K个模型。最后，将这K个模型的评估指标的平均值作为最终模型的评估指标。

具体步骤如下：

1. 将训练集分成K个子集，每个子集包含$\frac{1}{K}$的样本，记为$D=\{x_1, x_2, ..., x_m\}$，$x_i$为第$i$个样本。
2. 对于每个子集$D_i$，使用$D-D_i$作为训练集训练模型，使用$D_i$作为验证集计算模型的评估指标。
3. 重复第2步K次，得到K个模型和K个评估指标。
4. 计算K个评估指标的平均值，作为最终模型的评估指标。
5. 选择评估指标最优的模型作为最终模型。

## 1. cross_val_score

`cross_val_score` 是一个便捷的工具，用于执行交叉验证并返回每个测试折叠的评分（例如准确度、均方误差等）。它简化了交叉验证的过程，使你不必手动创建训练集和测试集。你只需提供模型、数据、评分指标和交叉验证策略即可。

- **使用示例**:
  ```python
  from sklearn.model_selection import cross_val_score
  scores = cross_val_score(estimator, X, y, cv=5, scoring='accuracy')
  ```

  这里，`estimator` 是一个模型实例（如 `LinearRegression()`、`RandomForestClassifier()` 等），`X` 和 `y` 是数据和标签，`cv` 指定了交叉验证的折叠数（在这个例子中是5），`scoring` 指定了评分标准（如 'accuracy'）。

## 2. KFold

`KFold` 提供了更多的灵活性。它是一个交叉验证分割器，将数据集分割为K个连续的折叠。与 `cross_val_score` 不同，`KFold` 仅仅是生成训练集和测试集索引的方法，它不直接进行模型训练和评分。

- **使用示例**:
  ```python
  from sklearn.model_selection import KFold
  kf = KFold(n_splits=5)

  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 训练模型
    model.fit(X_train, y_train)

    # 测试模型
    predictions = model.predict(X_test)

    # 计算并记录评估指标
    # ...
  ```

### 总结

- **cross_val_score**：更快捷、更简单，适用于快速评估模型性能。
- **KFold**：提供更高的灵活性，允许进行更复杂的操作，比如不同的数据预处理或参数调整。

选择使用哪一个工具取决于你的需求。如果你只需要快速评估模型的性能，`cross_val_score` 是一个很好的选择。如果你需要更复杂的数据处理或在不同的训练/测试分割上重复使用相同的数据集，那么 `KFold` 将是更好的选择。

## Cross Validation in selecting parameters
使用交叉验证来选择模型参数的步骤如下：

1. 选择一组参数值。
2. 对于每组参数值，执行K折交叉验证。
3. 选择平均评估指标最优的参数值。

### 使用cross_val_score选择KNN算法的最优参数

In [ ]:
# 交叉验证选择KNN算法的最优参数
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# 选择一组参数值
k_values = [1, 3, 5, 7, 9]

# 选择交叉验证策略
kfold = KFold(n_splits=5)

# 用于保存每组参数值的评估指标的平均值
kfold_scores = []

# 对于每组参数值，执行K折交叉验证
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=kfold, scoring='accuracy')
    kfold_scores.append(scores.mean())
    
# 选择平均评估指标最优的参数值,argmax返回最大值的索引
best_k = k_values[np.argmax(kfold_scores)]
print('Best k: {}'.format(best_k))

### 仅使用KFold选择KNN算法的最优参数

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold

# 选择一组K值
k_values = [1, 3, 5, 7, 9]

# 选择交叉验证策略
kfold = KFold(n_splits=5)

# 用于保存每组K值的评估指标的平均值
kfold_scores = []

# 对于每组K值，执行K折交叉验证，不用cross_val_score
for k in k_values:
    scores = []
    for train_index, test_index in kfold.split(X_train):
        # 训练模型
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train[train_index], y_train[train_index])
        # 测试模型
        predictions = knn.predict(X_train[test_index])
        # 计算并记录评估指标
        scores.append(accuracy_score(y_train[test_index], predictions))
        
    kfold_scores.append(np.mean(scores))

# 选择平均评估指标最优的K值
best_k = k_values[np.argmax(kfold_scores)]
print('Best k: {}'.format(best_k))

# 2. K-Nearest Neighbor Algorithm
## 1.算法基础
KNN算法，即K-最近邻算法（K-Nearest Neighbor Algorithm），是一种基础且广泛应用的分类和回归算法。其核心思想非常直观：通过测量不同特征值之间的距离来进行分类。

### KNN算法步骤：

1. **选择K值**：K是一个用户定义的常数，通常为小数。K值的选择会影响算法的结果。

2. **计算距离**：计算待分类点与其他所有点之间的距离。距离的计算方法可以是欧氏距离、曼哈顿距离等。

3. **找到最近的K个点**：根据计算出的距离，选择距离最近的K个点。

4. **进行投票或平均**：
   - **分类（Classification）**：这K个点中最常见的类别将被选为预测类别。
   - **回归（Regression）**：这K个点的目标值的平均值或中位数将被用作预测值。

### 特点：

- **优点**：
  - 简单易懂，易于实现。
  - 适合对稀有事件进行分类。
  - 在数据集足够大且没有噪声的情况下，KNN能够非常有效。

- **缺点**：
  - 计算成本高，特别是在大数据集上。
  - 对不平衡的数据集（某些类别的样本数量远多于其他类别）表现不佳。
  - 对于有很多特征（高维）的数据集，性能可能不佳（维度灾难）。
  - 需要事先确定K值，且K值的选择可能对结果有较大影响。

### 应用场景：

KNN算法广泛应用于推荐系统、图像识别和分类、搜索引擎中的相似项搜索等领域。但需要注意的是，它更适合数据量不是特别大的场景。

### Python实现：

在Python中，可以使用`scikit-learn`库中的`KNeighborsClassifier`或`KNeighborsRegressor`类来实现KNN算法。

```python
from sklearn.neighbors import KNeighborsClassifier

# 创建KNN分类器实例
knn = KNeighborsClassifier(n_neighbors=3)

# 使用数据训练模型
knn.fit(X_train, y_train)

# 进行预测
predictions = knn.predict(X_test)
```

KNN算法的效果很大程度上依赖于数据的质量和适当的K值选择。实际应用中，通常需要通过交叉验证等方法来选择最优的K值。

# 3.Naive Bayes Algorithm
## 1.算法基础
朴素贝叶斯算法（Naive Bayes Algorithm）是一种基于贝叶斯定理的简单且高效的分类算法。它的核心思想是假设特征之间相互独立，即使这个假设在现实中并不成立，但在实际应用中，朴素贝叶斯算法通常能够取得不错的效果。

### 朴素贝叶斯算法步骤：

1. **计算先验概率**：计算每个类别的先验概率。
2. **计算条件概率**：计算每个特征在每个类别下的条件概率。
3. **计算后验概率**：计算每个类别在给定特征下的后验概率。后验概率=先验概率×条件概率。
4. **进行分类**：选择具有最大后验概率的类别作为预测类别。 


## 2.代码实现
在Python中，可以使用`scikit-learn`库中的`GaussianNB`类来实现朴素贝叶斯算法。

```python
from sklearn.naive_bayes import GaussianNB

# 创建朴素贝叶斯分类器实例
gnb = GaussianNB()

# 使用数据训练模型
gnb.fit(X_train, y_train)

# 进行预测
predictions = gnb.predict(X_test)
``` 

# 4. Linear Regression
线性回归是一种用于预测和解释数据之间线性关系的统计方法。它是数据科学和机器学习领域中最基本和广泛使用的算法之一。

## 1. 基本概念：

线性回归模型试图建立自变量（解释变量、特征）和因变量（目标变量、响应变量）之间的线性关系。这种关系可以表示为一个线性方程，最简单的形式是一元线性回归，表示为：

$$\[ y = \theta_0 + \theta_1 x + \epsilon \]$$

其中：
1. $y$：因变量（目标变量、响应变量）
2. $x$：自变量（解释变量、特征）
3. $\theta_0$：截距
4. $\theta_1$：斜率
5. $\epsilon$：误差项
6. $\theta_0$和$\theta_1$：模型参数

### 多元线性回归：

当有多个自变量时，模型变为多元线性回归：

$$\[ y = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \ldots + \theta_n x_n + \epsilon \]$$

### 参数估计：

线性回归的目标是估计模型参数$（\( \theta \)）$，使得预测值尽可能接近实际观测值。通常使用最小二乘法来估计这些参数，即通过最小化误差项的平方和来找到最佳的参数值。

### 模型评估：

- **R²（决定系数）**：衡量模型对数据变异的解释程度。R²的取值范围为0到1，值越大表示模型对数据的解释程度越好。
- **均方误差（MSE）**：衡量预测值与实际值之间的差异。MSE的取值范围为0到正无穷，值越小表示模型预测的越准确。

### 在Python中的实现：

在Python中，可以使用`scikit-learn`库的`LinearRegression`类来实现线性回归：

```python
from sklearn.linear_model import LinearRegression

# 创建线性回归模型实例
model = LinearRegression()

# 训练模型
model.fit(X_train, y_train)

# 预测
predictions = model.predict(X_test)
```

### 注意事项：

- 线性回归假设数据之间存在线性关系。如果关系是非线性的，线性模型可能无法很好地拟合。
- 数据中的异常值会对模型参数估计产生显著影响。
- 需要检查多重共线性，即自变量之间高度相关的情况。


## 2. Polynomial Regression
多项式回归是一种特殊的线性回归，它通过将自变量的幂次方和交互项添加为自变量来扩展线性模型，从而拟合非线性数据。

### 多项式回归的一般形式：
$$\[ y = \theta_0 + \theta_1 x + \theta_2 x^2 + \theta_3 x^3 + \ldots + \theta_n x^n + \epsilon \]$$

### 在Python中的实现：
首先，需要使用`PolynomialFeatures`类将自变量转换为多项式特征，然后使用`LinearRegression`类来拟合转换后的数据。
```python
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# 创建多项式特征
poly_features = PolynomialFeatures(degree=2)

# 将X转换为多项式特征
X_train_poly = poly_features.fit_transform(X_train)

# 创建线性回归模型实例
model = LinearRegression()

# 训练模型
model.fit(X_train_poly, y_train)

# 预测
predictions = model.predict(X_test)
```

# 5. Basic Neural Network--Perceptron
## 1. Single Layer Perceptron
感知器（Perceptron）是一种最简单的神经网络模型，它是一种二分类模型，用于判断输入属于哪一类。单层感知机就是一个线性分类器，它通过计算输入和权重的线性组合，再加上偏置，然后将结果输入激活函数来得到输出。

### 感知器算法步骤：
输入：训练数据集$D=\{(x_1, y_1), (x_2, y_2), ..., (x_m, y_m)\}$，其中$x_i=(x_i^{(1)}, x_i^{(2)}, ..., x_i^{(n)})^T$，$x_i^{(j)}$是第$i$个样本的第$j$个特征，$y_i \in \{+1, -1\}$，$i=1,2,...,m$；学习率$\eta$（取值范围为0到1）。
输出：$w, b$；感知器模型$f(x)=sign(w \cdot x + b)$。
1. **初始化权重和偏置**：权重$w_0$和偏置$b_0$的初始值可以是随机值或者0。
2. **计算输出**：将线性组合的结果输入激活函数，得到输出。当激活函数为sign符号函数时，有$y=sign(w \cdot x + b)$。
3. **更新权重和偏置**：根据误差更新权重和偏置。如果预测正确，权重和偏置不变；如果预测错误，权重和偏置更新为$w=w+\eta y_i x_i$和$b=b+\eta y_i$。
4. **重复步骤2-4**：直到达到停止条件。

### 在Python中的实现：
在Python中，可以使用`scikit-learn`库中的`Perceptron`类来实现感知器算法。
```python
from sklearn.linear_model import Perceptron

# 创建感知器实例
model = Perceptron()
    
# 训练模型
model.fit(X_train, y_train)

# 预测
predictions = model.predict(X_test)
```


## 2. Multi-Layer Perceptron
多层感知器（Multi-Layer Perceptron，MLP）是一种前馈神经网络，它由一个输入层、一个或多个隐藏层和一个输出层组成。每个隐藏层由多个神经元组成，每个神经元都与上一层的所有神经元相连。多层感知器通过反向传播算法进行训练，可以用于分类和回归。

### 多层感知机代码实现：
在Python中，可以使用`scikit-learn`库中的`MLPClassifier`类来实现多层感知机算法。
```python
from sklearn.neural_network import MLPClassifier

# 创建多层感知机实例
model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=400)

# 训练模型
model.fit(X_train, y_train)

# 预测
predictions = model.predict(X_test)
```

其中，`hidden_layer_sizes=(x,y,z, ...)` 表示隐藏层的神经元个数，x表示第一层隐藏层的神经元个数，y表示第二层隐藏层的神经元个数，z表示第三层隐藏层的神经元个数

`max_iter`表示最大迭代次数

# 6. Decision Tree
## 1.算法基础
决策树（Decision Tree）是一种基本的分类和回归方法，它通过训练数据构建树形结构，用于预测数据的性质或目标变量。决策树的优点是易于理解和解释，可以可视化，需要很少的数据准备。但是，决策树很容易产生过拟合问题。

### 决策树算法步骤：
1. **选择最优特征**：使用特征选择算法选择最优特征。
2. **划分训练集**：根据选择的特征，将训练集划分为子集。
3. **生成决策树**：递归地生成决策树。
4. **进行剪枝**：使用剪枝算法剪枝。
5. **预测**：对测试集进行预测。

### 决策树的优点：
- 决策树易于理解和解释，可以可视化。
- 对于决策树，数据的准备工作一般比较简单。一般不需要对数据进行归一化处理，也不需要处理缺失值。
- 能够同时处理数据型和常规型属性。
- 决策树是一个白盒模型。给定一个观察的模型，很容易推出相应的逻辑表达式。

### 决策树的缺点：
- 决策树学习的过程容易产生过拟合问题。
- 忽略了数据之间的相关性。
- 对于那些各类别样本数量不一致的数据，在决策树中，信息增益的结果偏向于那些具有更多数值的特征。

### 代码实现：
在Python中，可以使用`scikit-learn`库中的`DecisionTreeClassifier`类来实现决策树分类算法。
`DesionTreeRegressor`类用于回归问题。两者的参数和使用方法基本相同。
```python
from sklearn.tree import DecisionTreeClassifier

# 创建决策树实例
model = DecisionTreeClassifier()
    
# 训练模型
model.fit(X_train, y_train)

# 预测
predictions = model.predict(X_test)
```


### 决策树可视化
#### 1. text representation
可以使用 sklearn.tree中的`export_text`生成文本树描述

```
from sklearn.tree import export_text
r = export_text(model,feature_names=features)
print(r)
```

#### 2. graphical representation
使用`plot_tree`函数生成决策树的可视化图形
```
from sklearn.tree import plot_tree
plt.figure(figsize=(20,10))
plot_tree(model,feature_names=features,class_names=target_names,filled=True,fontsize=10)
plt.show()
```



In [ ]:
# Extract a text-based representation
# export_text 用于生成决策树的文本描述，参数为：决策树模型，特征名称

from sklearn.tree import export_text 
features=breast_cancer['feature_names'].tolist()
r = export_text(model,feature_names=features) 
print(r)

## 2. Random Forest
随机森林（Random Forest）是一种集成学习方法，属于Bagging算法的一种。它通过集成多个决策树来完成分类和回归任务。随机森林的优点是易于实现，计算开销小，能够处理很高维度的数据，并且能够评估各个特征在分类问题上的重要性。

优点：
- 计算开销小，速度快。
- 在训练过程中，能够检测到特征间的互相影响。
- 能够评估各个特征在分类问题上的重要性。
- 在生成过程中，能够获取到对于泛化性能有帮助的内部结构信息。
- 对于缺省值问题不敏感。
- 能够处理很高维度（feature很多）的数据，并且不用做特征选择。

缺点：
- 随机森林在某些噪音较大的分类或回归问题上会过拟合。
- 参数调整比较困难。

算法步骤：
1. **随机选择样本**：从训练集中随机选择样本，构建子训练集。
2. **随机选择特征**：从所有特征中随机选择部分特征，构建子特征集。
3. **生成决策树**：使用子训练集和子特征集生成决策树。
4. **重复步骤1-3**：重复步骤1-3，直到生成足够多的决策树。
5. **进行预测**：对于分类问题，使用投票法确定最终类别；对于回归问题，使用平均法确定最终预测值。

代码实现：
在Python中，可以使用`scikit-learn`库中的`RandomForestClassifier`类来实现随机森林算法。
```python
from sklearn.ensemble import RandomForestClassifier

# 创建随机森林实例
model = RandomForestClassifier(n_estimators=100)

# 训练模型
model.fit(X_train, y_train)

# 预测
predictions = model.predict(X_test)
```

其中RandomForestClassifier类的参数如下：
- **n_estimators**：森林中树的数量。
- **criterion**：用于衡量分割质量的函数。支持的标准有"gini"代表基尼系数和"entropy"代表信息增益。默认值为"gini"。
- **max_depth**：树的最大深度。默认值为"None"，表示节点会一直扩展直到所有叶子都是纯的或者所有叶子节点都包含少于min_samples_split个样本。
- **min_samples_split**：分割内部节点所需的最少样本数。默认值为2。
- **min_samples_leaf**：叶子节点所需的最小样本数。默认值为1。 